In [ ]:
# run this for working with colab
import torch.cuda as cuda
cuda.get_device_name()

from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/lab/SSD/"
dir = "/content/gdrive/My Drive/lab/SSD/"


In [1]:
import os
import torch
import torch.nn as nn
import torch.cuda as cuda
from torch.utils.data import DataLoader, random_split
from dataset import VOCDataset
#from dataset_h5 import VOCDataset
from model import SSD300
from utils import *

device = torch.device("cuda:0" if cuda.is_available() else "cpu") 

## Load dataset

In [1]:
#from utils import create_data_lists
voc07_path = 'D:/C-dir/SSD/VOCdevkit/VOC2007/'
voc12_path = 'D:/C-dir/SSD/VOCdevkit/VOC2012/'
create_data_lists(voc07_path, voc12_path, 'D:/C-dir/SSD/VOCdevkit/')


There are 21503 training images containing a total of 62199 objects. Files have been saved to D:\C-dir\SSD\VOCdevkit.

There are 5138 test images containing a total of 7330 objects. Files have been saved to D:\C-dir\SSD\VOCdevkit.


In [49]:
from utils import *

In [2]:
torch.manual_seed(42)
trainset = VOCDataset(data_folder='datasets/', is_trainset=True, transform=True)
train_size = int(0.9 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = random_split(trainset, [train_size, val_size])
testset = VOCDataset(data_folder='datasets/', is_trainset=False, transform=False)

dataloaders = dict(
    train = DataLoader(trainset, batch_size=16, shuffle=True, num_workers=0),
    val = DataLoader(valset, batch_size=64, shuffle=False, num_workers=0),
    test = DataLoader(testset, batch_size=64, shuffle=False, num_workers=0)
)

## Define model

In [3]:
#vgg16_url = "https://download.pytorch.org/models/vgg16-397923af.pth"
#vgg16 = torch.hub.load_state_dict_from_url(vgg16_url, model_dir='models/')

In [ ]:
MySSD300 = SSD300(n_classes = 21, )

## Train model

In [ ]:
# Pre define training params

checkpoint =
lr = None
weight_decay = 5e-4
loss_func = MultiBoxLoss(priors_cxcy = MySSD300.get_prior_boxes(), threshold=0.5, neg_pos_ratio=3, alpha=1.)
optimizer = torch.optim.Adam(MySSD300.parameters())
grad_clip = None


def train_epoch(model, dataloaders):
    model.train()
    epoch_loss = 0
    for step, (imgs, boxes, labels,_) in enumerate(dataloaders['train']):
        imgs, boxes, labels = imgs.to(device), boxes.to(device), labels.to(device)
        offsets, class_scores = model(imgs)
        loss = loss_func(offsets, class_scores, boxes, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print('Step:', step, '- Done - loss:', loss.item())
    return epoch_loss/len(dataloaders['train'])

def eval(model, dataloaders):
    model.eval()
    epoch_loss = 0
    for x, y in dataloaders['val']:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = loss_func(pred, y)
        epoch_loss += loss.item()
        
        #calculate accuracy
        
        
    loss = total_loss/len(dataloaders['val'])
    
    
    return loss, accuracy


def train_model(model, dataloaders, epoch=1, ):
    
    if checkpoint is None:
    
    save_checkpoint(epoch, model, optimizer)

In [ ]:
#Try
train_epoch(MySSD300, dataloaders)
eval_epoch(MySSD300, dataloaders)
